# Pipeline 

In [ ]:
import pandas as pd
import numpy as np
import timeseriesflattener

### Loading data


In [ ]:
df_female = pd.read_csv('test_data/df_female.csv')
df_age = pd.read_csv('test_data/df_age.csv')
df_SFI = pd.read_csv('test_data/df_SFI.csv')
df_uti = pd.read_csv('test_data/df_uti.csv')
df_admissions = pd.read_csv('test_data/df_admissions.csv')

#REmoving extra column
df_female = df_female.loc[:, ~df_female.columns.str.contains('^Unnamed')]
df_age = df_age.loc[:, ~df_age.columns.str.contains('^Unnamed')]
df_SFI = df_SFI.loc[:, ~df_SFI.columns.str.contains('^Unnamed')]
df_uti = df_uti.loc[:, ~df_uti.columns.str.contains('^Unnamed')]
df_admissions = df_admissions.loc[:, ~df_admissions.columns.str.contains('^Unnamed')]


In [ ]:
df_female
df_SFI
df_uti
df_admissions

In [ ]:
#Creating uti df with negative cases as well
merged = pd.merge(df_age, df_uti, on=['ID'], how='left')

merged = merged[['ID','value']]

#turning UVI-neg cases from nan to 0
where_are_NaNs = np.isnan(merged)
merged[where_are_NaNs] = 0

#checking
print(merged)

#Changing from floats to int
merged['value'] = merged['value'].astype('int')

#Creating subsets
uti_full = merged

uti_value = merged['value']
print(uti_value)

ID = merged['ID']
ID

### Data split

In [ ]:
from sklearn.model_selection import train_test_split

X = ID #The ID column
y = uti_value #The outcome column


# # Uncomment for a different data split: 60% training, 20% validation, 20% test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
# X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.80 = 0.2


#70% training, 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)



In [ ]:
print(f"Number of observations in trainingset:{sum(y_train)}") #354
print(f"Percentage of positive class in trainingset: {sum(y_train)/len(y_train)*100}") #27.15% UTI


print(f"Number of observations in testset:{sum(y_test)}") #152
print(f"Percentage of positive class in testset: {sum(y_test)/len(y_test)*100}") #27.14% UTI

In [ ]:
#print(X_train)

#print(len(X_train.unique()))
#print(len(ID.unique()))

## Creating a df for uti positive within the training set 
#NB 'value' = uti
train_data = pd.DataFrame({'ID' : X_train, 'value' : y_train})

print(train_data.head())

train_data_sub = train_data[train_data['value'] == 1]

y_train_df = pd.merge(train_data_sub, df_uti, on=['ID', 'value'], how='left')

print(len(train_data_sub) == len(y_train_df))

print(y_train_df.head())

df_uti.head()

In [ ]:
## Creating a df for uti positive within the test set 
#NB 'value' = uti
test_data = pd.DataFrame({'ID' : X_test, 'value' : y_test})

#print(test_data.head())

#Subset with only uti-pos
test_data_sub = test_data[test_data['value'] == 1]

#adding the dates from the uti df
y_test_df = pd.merge(test_data_sub, df_uti, on=['ID', 'value'], how='left')

#Checking nothing went wrong
print(len(test_data_sub) == len(y_test_df))

print(y_test_df.head())

df_uti.head()

In [ ]:
#TESTING 12/02

X_train_df = pd.DataFrame({'ID' : X_train})

predictor_df_dict = {'df_SFI':df_SFI, 'df_age':df_age, 'df_female':df_female}

d = {}

In [ ]:
#TESTING 12/02
for key in predictor_df_dict:
    d[key] = pd.merge(X_train_df, predictor_df_dict[key], on=['ID'], how='left')


In [ ]:
#TESTING 12/02
d['df_SFI']

In [ ]:

X_train_df = pd.DataFrame({'ID' : X_train})

X_train_df

print(len(df_SFI))

X_train_SFI = pd.merge(X_train_df, df_SFI, on=['ID'], how='left')
X_train_age = pd.merge(X_train_df, df_age, on=['ID'], how='left')
X_train_female = pd.merge(X_train_df, df_female, on=['ID'], how='left')

X_train_SFI

In [ ]:
#Merging the different test datasets
X_test_df = pd.DataFrame({'ID' : X_test})

X_test_df

X_test_SFI = pd.merge(X_test_df, df_SFI, on=['ID'], how='left')
X_test_age = pd.merge(X_test_df, df_age, on=['ID'], how='left')
X_test_female = pd.merge(X_test_df, df_female, on=['ID'], how='left')

#Checking everything worked
X_test_SFI
#print(X_test_age.head())
#print(X_test_gender.head())

In [ ]:
#Creating a train subset of df_admissions based on the ID's from X_train
ID_list_train = list(X_train)

ID_list_train

df_admissions_train = df_admissions[df_admissions['ID'].isin(ID_list_train)]

df_admissions #80876 rows
df_admissions_train #56271 rows

In [ ]:
#Creating a test subset of df_admissions based on the ID's from X_test
ID_list_test = list(X_test)

ID_list_test

df_admissions_test = df_admissions[df_admissions['ID'].isin(ID_list_test)]

df_admissions #80876 rows
df_admissions_test #24605 rows #NB lidt mere end 30% fordi der er lidt flere indlæggelsesdage pr patient her.

In [ ]:
boolarr = X_train_SFI['ID'].unique() == X_train_df['ID']

print(boolarr.sum() == len(boolarr)) #If true, they are all equal

X_train_SFI

In [ ]:
boolarr = X_test_SFI['ID'].unique() == X_test_df['ID']

print(boolarr.sum() == len(boolarr)) #If true, they are all equal

X_test_SFI

### Sentence transformer

In [ ]:
#For removing numbers in notes.... Dont know if this should be applied or not....

X_train_SFI['note'] = X_train_SFI['note'].str.replace('\d+', '')
X_test_SFI['note'] = X_test_SFI['note'].str.replace('\d+', '')

In [ ]:
from sentence_transformers import SentenceTransformer

#sentence_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2') #is this the one I wanna use??

sentence_model = SentenceTransformer('encoder-large-v1')
sentence_model

sentence_embeddings_train = sentence_model.encode(X_train_SFI['note'])
sentence_embeddings_test = sentence_model.encode(X_test_SFI['note'])

In [ ]:
sentence_embeddings_train_df = pd.DataFrame(sentence_embeddings_train)
sentence_embeddings_train_df # 93607 rows × 1024 columns
X_train_SFI #93607 rows × 3 columns
#X_train_df

sentence_embeddings_test_df = pd.DataFrame(sentence_embeddings_test)
sentence_embeddings_test_df # 39478 rows × 1024 columns
X_test_SFI #39478 rows × 3 columns              #Bare lige et tjek for at de rent faktisk er samme størrelse

In [ ]:
X_train_embeddings = pd.merge(X_train_SFI, sentence_embeddings_train_df, how='left', left_index=True, right_index=True)
X_train_embeddings 

df_sentence_train = X_train_embeddings.loc[:, X_train_embeddings.columns != 'note']

df_sentence_train 



X_test_embeddings = pd.merge(X_test_SFI, sentence_embeddings_test_df, how='left', left_index=True, right_index=True)
X_test_embeddings 

df_sentence_test = X_test_embeddings.loc[:, X_test_embeddings.columns != 'note']

df_sentence_test 

In [ ]:
#Tester cosine similarity, bare for at se om embeddingsene er okay

# for sentence, embedding in zip(X_train_SFI['note'], sentence_embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

# from sentence_transformers import SentenceTransformer, util

# cos_sim1 = util.cos_sim(sentence_embeddings[93603], sentence_embeddings[93605])
# print("Cosine-Similarity:", cos_sim1) #Similar sentences (våd ble, uridom påsat     Våd ble skiftet, uridom på) cosine = 0.9

# cos_sim2 = util.cos_sim(sentence_embeddings[3], sentence_embeddings[93605])
# print("Cosine-Similarity:", cos_sim2) #UNsimilar sentences (ingen lugt    Våd ble skiftet, uridom på) cosine = 0.68


# #X_train_embeddings = pd.merge(X_train_SFI, sentence_embeddings_df, how='left')


### Making tf-idf-data ready for flattening

In [ ]:
from timeseriesflattener.df_transforms import (
    df_with_multiple_values_to_named_dataframes,
)

# split the dataframe into a list of named dataframes with one value each
embedded_dfs_train = df_with_multiple_values_to_named_dataframes(
    df=df_sentence_train,
    entity_id_col_name="ID",
    timestamp_col_name="date",
    name_prefix="tfidf_",
)

# check the first dataframe
embedded_dfs_train[0].df.head()

In [ ]:
from timeseriesflattener.df_transforms import (
    df_with_multiple_values_to_named_dataframes,
)

# split the dataframe into a list of named dataframes with one value each
embedded_dfs_test = df_with_multiple_values_to_named_dataframes(
    df=df_sentence_test,
    entity_id_col_name="ID",
    timestamp_col_name="date",
    name_prefix="tfidf_",
)

# check the first dataframe
embedded_dfs_test[0].df.head()

In [ ]:
from timeseriesflattener.aggregation_fns import mean
from timeseriesflattener.feature_specs.group_specs import PredictorGroupSpec
import numpy as np

# create a group spec for the embedded text that will take the mean of each embedding on the column axis
# for the last 365 and 730 days
emb_spec_batch_train = PredictorGroupSpec(
    named_dataframes=embedded_dfs_train,
    lookbehind_days=[4], #kan tilføje flere [7, 14] så for man en predictor som kigger 7 dage tilbage og en der kigger 14 dage tilbage
    fallback=[np.nan],
    aggregation_fns=[mean],
).create_combinations()

# print the number of features we will create
print(len(emb_spec_batch_train))

In [ ]:
from timeseriesflattener.aggregation_fns import mean
from timeseriesflattener.feature_specs.group_specs import PredictorGroupSpec
import numpy as np

# create a group spec for the embedded text that will take the mean of each embedding on the column axis
# for the last 365 and 730 days
emb_spec_batch_test = PredictorGroupSpec(
    named_dataframes=embedded_dfs_test,
    lookbehind_days=[4], #kan tilføje flere [7, 14] så for man en predictor som kigger 7 dage tilbage og en der kigger 14 dage tilbage
    fallback=[np.nan],
    aggregation_fns=[mean],
).create_combinations()

# print the number of features we will create
print(len(emb_spec_batch_test))

In [ ]:
df_admissions_train2 = df_admissions_train.reset_index(drop=True)
df_admissions_train2 
df_admissions_train

In [ ]:
df_admissions_test2 = df_admissions_test.reset_index(drop=True)

df_admissions_test2 
df_admissions_test

### Timeseriesflattener

In [ ]:
if __name__ == "__main__":

    # Load a dataframe with times you wish to make a prediction
    prediction_times_df = df_admissions_train2
    # Load a dataframe with raw values you wish to aggregate as predictors
    #predictor_df =  # Using my embeddings above
    # Load a dataframe specifying when the outcome occurs
    outcome_df = y_train_df

    # Specify how to aggregate the predictors and define the outcome
    from timeseriesflattener.feature_specs.single_specs import OutcomeSpec, PredictorSpec, StaticSpec
    from timeseriesflattener.aggregation_fns import maximum, mean

    sex_predictor_spec_train = StaticSpec(
       timeseries_df= X_train_female,
       feature_base_name="female",
       prefix="pred",
     )
    
    age_predictor_spec_train = StaticSpec(
       timeseries_df= X_train_age,
       feature_base_name="age",
       prefix="pred",
     )

    outcome_spec_train = OutcomeSpec(
        timeseries_df=outcome_df,
        lookahead_days=3,
        fallback=0,
        aggregation_fn=maximum,
        feature_base_name="UTI",
        incident=False,
    )

    # Instantiate TimeseriesFlattener and add the specifications
    from timeseriesflattener import TimeseriesFlattener

    ts_flattener_train = TimeseriesFlattener(
        prediction_times_df=prediction_times_df,
        entity_id_col_name="ID",
        timestamp_col_name="date",
        n_workers=40,
        drop_pred_times_with_insufficient_look_distance=True,
    )
    ts_flattener_train.add_spec([sex_predictor_spec_train, outcome_spec_train, *emb_spec_batch_train, age_predictor_spec_train]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
    df_train = ts_flattener_train.get_df()
    df_train

In [ ]:
# adding additional data

df_FIM_total = pd.read_csv('test_data2/df_FIM_total.csv')
df_acute_days = pd.read_csv('test_data2/df_acute_days.csv')
df_brain_injury = pd.read_csv('test_data2/df_brain_injury_mod.csv')

df_FIM_total = df_FIM_total.loc[:, ~df_FIM_total.columns.str.contains('^Unnamed')]
df_acute_days = df_acute_days.loc[:, ~df_acute_days.columns.str.contains('^Unnamed')]
df_brain_injury = df_brain_injury.loc[:, ~df_brain_injury.columns.str.contains('^Unnamed')]

X_train_brain_injury = pd.merge(X_train_df, df_brain_injury, on=['ID'], how='left')
X_train_acute_days = pd.merge(X_train_df, df_acute_days, on=['ID'], how='left')
X_train_FIM_total = pd.merge(X_train_df, df_FIM_total, on=['ID'], how='left')


# X_train_acute_days[' acute_days'] = X_train_acute_days[' acute_days'].astype(int)
# print(X_train_brain_injury)
# print(X_train_acute_days)
# print(X_train_FIM_total)
# print(X_train_acute_days.dtypes)
# print(np.unique(X_train_acute_days[' acute_days']))

X_test_brain_injury = pd.merge(X_test_df, df_brain_injury, on=['ID'], how='left')
X_test_acute_days = pd.merge(X_test_df, df_acute_days, on=['ID'], how='left')
X_test_FIM_total = pd.merge(X_test_df, df_FIM_total, on=['ID'], how='left')


#rename FIM_total to value (column-name needs to be value for timeseriesflattener)
X_train_FIM_total = X_train_FIM_total.rename(columns= {'FIM_total': 'value'})

#rename FIM_total to value (column-name needs to be value for timeseriesflattener)
X_test_FIM_total = X_test_FIM_total.rename(columns= {'FIM_total': 'value'})



In [ ]:

acute_days_predictor_spec_train = StaticSpec(
   timeseries_df= X_train_acute_days,
   feature_base_name="acute_days",
   prefix="pred" )


ts_flattener_train.add_spec([acute_days_predictor_spec_train]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
df_train = ts_flattener_train.get_df()
df_train

In [ ]:

brain_injury_predictor_spec_train = StaticSpec(
   timeseries_df= X_train_brain_injury,
   feature_base_name="brain_injury",
   prefix="pred")


ts_flattener_train.add_spec([brain_injury_predictor_spec_train ]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
df_train = ts_flattener_train.get_df()
df_train
    
# FIM_total_predictor_spec_train = PredictorSpec(
#    timeseries_df= X_train_FIM_total,
#    lookbehind_days=4,
#    fallback=np.nan, 
#    aggregation_fn=mean,
#    feature_base_name="FIM_total") #prefix="pred" Sker automatisk fordi det er en PredictorSpec



In [ ]:

FIM_total_predictor_spec_train = PredictorSpec(
    timeseries_df= X_train_FIM_total,
    lookbehind_days=4,
    fallback=np.nan, 
    aggregation_fn=mean,
    feature_base_name="FIM_total") #prefix="pred" Sker automatisk fordi det er en PredictorSpec

ts_flattener_train.add_spec([FIM_total_predictor_spec_train]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
df_train = ts_flattener_train.get_df()
df_train
    


In [ ]:
#df_train.to_csv("test_data/testing_df_X_train.csv")

df_train.dropna().head()

In [ ]:
if __name__ == "__main__":

    # Load a dataframe with times you wish to make a prediction
    prediction_times_df = df_admissions_test2
    # Load a dataframe with raw values you wish to aggregate as predictors
    #predictor_df =  # Using my embeddings above
    # Load a dataframe specifying when the outcome occurs
    outcome_df = y_test_df

    # Specify how to aggregate the predictors and define the outcome
    from timeseriesflattener.feature_specs.single_specs import OutcomeSpec, PredictorSpec, StaticSpec
    from timeseriesflattener.aggregation_fns import maximum, mean

    sex_predictor_spec_test = StaticSpec(
       timeseries_df= X_test_female,
       feature_base_name="female",
       prefix="pred",
     )
    
    age_predictor_spec_test = StaticSpec(
       timeseries_df= X_test_age,
       feature_base_name="age",
       prefix="pred",
     )

    outcome_spec_test = OutcomeSpec(
        timeseries_df=outcome_df,
        lookahead_days=3,
        fallback=0,
        aggregation_fn=maximum,
        feature_base_name="UTI",
        incident=False,
    )

    # Instantiate TimeseriesFlattener and add the specifications
    from timeseriesflattener import TimeseriesFlattener

    ts_flattener_test = TimeseriesFlattener(
        prediction_times_df=prediction_times_df,
        entity_id_col_name="ID",
        timestamp_col_name="date",
        n_workers=40,
        drop_pred_times_with_insufficient_look_distance=True,
    )
    ts_flattener_test.add_spec([sex_predictor_spec_test, outcome_spec_test, *emb_spec_batch_test, age_predictor_spec_test]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
    df_test = ts_flattener_test.get_df()
    df_test

In [ ]:
acute_days_predictor_spec_test = StaticSpec(
   timeseries_df= X_test_acute_days,
   feature_base_name="acute_days",
   prefix="pred" )


ts_flattener_test.add_spec([acute_days_predictor_spec_test]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
df_test = ts_flattener_test.get_df()
df_test

In [ ]:
brain_injury_predictor_spec_test = StaticSpec(
   timeseries_df= X_test_brain_injury,
   feature_base_name="brain_injury",
   prefix="pred")


ts_flattener_test.add_spec([brain_injury_predictor_spec_test ]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
df_test = ts_flattener_test.get_df()
df_test

In [ ]:
FIM_total_predictor_spec_test= PredictorSpec(
    timeseries_df= X_test_FIM_total,
    lookbehind_days=4,
    fallback=np.nan, 
    aggregation_fn=mean,
    feature_base_name="FIM_total") #prefix="pred" Sker automatisk fordi det er en PredictorSpec

ts_flattener_test.add_spec([FIM_total_predictor_spec_test]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
df_test = ts_flattener_test.get_df()
df_test
    

In [ ]:
# #Added 13/02. ADDING PSYK_group SFI DATA (1/2) NB:tfidf embeddings

# #Model is not better with this data, but it is far slower


# df_SFI_psyk = pd.read_csv('test_data2/df_SFI_psyk.csv')

# df_SFI_psyk = df_SFI_psyk.loc[:, ~df_SFI_psyk.columns.str.contains('^Unnamed')]


# X_train_SFI_psyk = pd.merge(X_train_df, df_SFI_psyk, on=['ID'], how='left')
# X_test_SFI_psyk = pd.merge(X_test_df, df_SFI_psyk, on=['ID'], how='left')

# X_train_SFI_psyk = X_train_SFI_psyk.dropna()

# X_test_SFI_psyk = X_test_SFI_psyk.dropna()


# X_train_SFI_psyk['note'] = X_train_SFI_psyk['note'].str.replace('\d+', '')
# X_test_SFI_psyk['note'] = X_test_SFI_psyk['note'].str.replace('\d+', '')

# from sklearn.feature_extraction.text import TfidfVectorizer
# from spacy.lang.da.stop_words import STOP_WORDS


# danish_stopwords = list(STOP_WORDS)

# vectorizer = TfidfVectorizer(stop_words = danish_stopwords, min_df = 5, max_features=200) #with min_df = 2, 7676 tokens. without 11314 tokens, with min_df = 5, 4031 tokens, min_df = 100. 622 tokens
# #Really bad results with min_df = 5 and max_features = 200, i.e. highest of cross-val-score was about 0.65, lowest about 0.35
# #same for min_df = 5 and max_features = 500
# #same for min_df = 5 and max_features = 1000 
# #same for min_df = 20 and max_features = 500

# X_train_psyk = vectorizer.fit_transform(X_train_SFI_psyk['note'])

# #print("Tokens", vectorizer.get_feature_names_out())

# tokens_psyk = vectorizer.get_feature_names_out()

# print(X_train_psyk.shape) 

# X_train_psyk
# # Convert sparse matrix to DataFrame
# df_tfidfvect_train_psyk = pd.DataFrame(data = X_train_psyk.toarray(), columns = tokens_psyk)
# df_tfidfvect_train_psyk


# #Running the model on the test dataset NB using transform() instead of fit transform, in order to avoid training the model on the  test set
# X_test_psyk = vectorizer.transform(X_test_SFI_psyk['note'])

# ##### DETTE SKAL VEL IKKE VÆRE I TEST PIPELINEN?
# print("Tokens", vectorizer.get_feature_names_out())

# tokens_psyk = vectorizer.get_feature_names_out()

# print(X_test_psyk.shape) 

# X_test_psyk
# # Convert sparse matrix to DataFrame
# df_tfidfvect_test_psyk = pd.DataFrame(data = X_test_psyk.toarray(), columns = tokens_psyk) #bruger tokens fra X_train
# df_tfidfvect_test_psyk

# #Adding date and ID to the train df
# df_tfidfvect_train_psyk['ID'] = X_train_SFI_psyk['ID']
# df_tfidfvect_train_psyk['date'] = X_train_SFI_psyk['date']

# #Checking 
# df_tfidfvect_train_psyk

# #Adding date and ID to the train df
# df_tfidfvect_test_psyk['ID'] = X_test_SFI_psyk['ID']
# df_tfidfvect_test_psyk['date'] = X_test_SFI_psyk['date']

# #Checking 
# df_tfidfvect_test_psyk

In [ ]:
# #Added 13/02 ADDING PSYK_group SFI DATA (2/2) NB:tfidf embeddings

# from timeseriesflattener.df_transforms import (
#     df_with_multiple_values_to_named_dataframes,
# )

# # split the dataframe into a list of named dataframes with one value each
# embedded_dfs_train_psyk = df_with_multiple_values_to_named_dataframes(
#     df=df_tfidfvect_train_psyk,
#     entity_id_col_name="ID",
#     timestamp_col_name="date",
#     name_prefix="tfidf_",
# )

# # check the first dataframe
# embedded_dfs_train_psyk[0].df.head()


# # split the dataframe into a list of named dataframes with one value each
# embedded_dfs_test_psyk = df_with_multiple_values_to_named_dataframes(
#     df=df_tfidfvect_test_psyk,
#     entity_id_col_name="ID",
#     timestamp_col_name="date",
#     name_prefix="tfidf_",
# )

# # check the first dataframe
# embedded_dfs_test_psyk[0].df.head()


# from timeseriesflattener.aggregation_fns import mean
# from timeseriesflattener.feature_specs.group_specs import PredictorGroupSpec
# import numpy as np

# # create a group spec for the embedded text that will take the mean of each embedding on the column axis
# # for the last 365 and 730 days
# emb_spec_batch_train_psyk = PredictorGroupSpec(
#     named_dataframes=embedded_dfs_train_psyk,
#     lookbehind_days=[4], #kan tilføje flere [7, 14] så for man en predictor som kigger 7 dage tilbage og en der kigger 14 dage tilbage
#     fallback=[np.nan],
#     aggregation_fns=[mean],
# ).create_combinations()

# # print the number of features we will create
# print(len(emb_spec_batch_train_psyk))



# # create a group spec for the embedded text that will take the mean of each embedding on the column axis
# # for the last 365 and 730 days
# emb_spec_batch_test_psyk = PredictorGroupSpec(
#     named_dataframes=embedded_dfs_test_psyk,
#     lookbehind_days=[4], #kan tilføje flere [7, 14] så for man en predictor som kigger 7 dage tilbage og en der kigger 14 dage tilbage
#     fallback=[np.nan],
#     aggregation_fns=[mean],
# ).create_combinations()

# # print the number of features we will create
# print(len(emb_spec_batch_test_psyk))


# ts_flattener_train.add_spec([*emb_spec_batch_train_psyk]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
# df_train = ts_flattener_train.get_df()
# df_train


# ts_flattener_test.add_spec([*emb_spec_batch_test_psyk]) #Stjerne for at unpack listen, yderligere predictors tilføjes blot til listen
    
# df_test = ts_flattener_test.get_df()
# df_test

## Data split

NEXT STEPS

impute (måde og handle missing), prøv xgboost (som selv dealer med missing)

fjern dublicates i dataset, check

evt merge nye specs på ved prediction_time_uuid, check

sklearn crossvalidator_score(), check

Lav admissions_df hvor admission times er i hhv train og test 

In [ ]:
#df_test.to_csv("test_data/testing_df_X_test.csv")

df_test.dropna().head()
df_test.head()


## Training ML model

In [ ]:
from scipy.stats import uniform, randint

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict


#df= timeseriesflattenener.get_df([specs]) # koden fra sidst

xgb_model = XGBClassifier(objective="binary:logistic", random_state=42, enable_categorical = True, tree_method = 'hist')
#xgb_model = XGBClassifier(objective="binary:logistic", random_state=42, enable_categorical = True, tree_method = 'hist', reg_lambda = 10, reg_alpha = 2) #Virker fint, men gør ikke rigtig nogen forskel
#xgb_model = XGBClassifier(objective="binary:logistic", class_weight = 'balanced', random_state=42, enable_categorical = True, tree_method = 'hist')

# find alle predictor columns (har præfix "pred_")
predictor_columns_train =  [col for col in df_train if col.startswith('pred_')]

# find outcome column (har præfix "outc_")
outcome_column_train = [col for col in df_train if col.startswith('outc_')]

# find metadata columns (fx prediction_time_uuid)
metadata_columns_train = [col for col in df_train if col not in predictor_columns_train + outcome_column_train]


# lav X der kun er predictors, og y der kun er outcome
X_final_train = df_train[predictor_columns_train]
y_final_train = df_train[outcome_column_train]

X_final_train.to_csv("test_data/X_final_train1.csv") 
y_final_train.to_csv("test_data/y_final_train1.csv")


In [ ]:

# find alle predictor columns (har præfix "pred_")
predictor_columns_test =  [col for col in df_test if col.startswith('pred_')]

# find outcome column (har præfix "outc_")
outcome_column_test = [col for col in df_test if col.startswith('outc_')]

# find metadata columns (fx prediction_time_uuid)
metadata_columns_test = [col for col in df_test if col not in predictor_columns_test + outcome_column_test]


# lav X der kun er predictors, og y der kun er outcome
X_final_test = df_test[predictor_columns_test]
y_final_test = df_test[outcome_column_test]

X_final_test.to_csv("test_data/X_final_test1.csv")
y_final_test.to_csv("test_data/y_final_test1.csv")

## Start here to avoid rerunning everything after crash

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict


#df= timeseriesflattenener.get_df([specs]) # koden fra sidst. NB stavet forkert, hvis det ikke virker er det nok grunden

#xgb_model = XGBClassifier(objective="binary:logistic", random_state=42, enable_categorical = True, tree_method = 'hist')
xgb_model = XGBClassifier(objective="binary:logistic", random_state=42, enable_categorical = True, tree_method = 'hist', reg_alpha = 20)
#xgb_model = XGBClassifier(objective="binary:logistic", random_state=42, enable_categorical = True, tree_method = 'hist', reg_lambda = 100) #, reg_alpha = 30) #Virker fint, men gør ikke rigtig nogen forskel


X_final_train = pd.read_csv('test_data/X_final_train1.csv') #X_final_train = med tal i tf-idf, X_final_train1 = ingen tal i tf-idf
y_final_train = pd.read_csv('test_data/y_final_train1.csv')

X_final_test = pd.read_csv('test_data/X_final_test1.csv')
y_final_test = pd.read_csv('test_data/y_final_test1.csv')

#REmoving extra column
X_final_train = X_final_train.loc[:, ~X_final_train.columns.str.contains('^Unnamed')]
y_final_train = y_final_train.loc[:, ~y_final_train.columns.str.contains('^Unnamed')]
X_final_test = X_final_test.loc[:, ~X_final_test.columns.str.contains('^Unnamed')]
y_final_test = y_final_test.loc[:, ~y_final_test.columns.str.contains('^Unnamed')]

In [ ]:
X_final_test

In [ ]:
# Oversampling 
from imblearn.over_sampling import RandomOverSampler #, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler #,ClusterCentroids

ros = RandomOverSampler(random_state=42) #, shrinkage = 0.5) #Can't use shrinkage. When shrinkage is used, X needs to contain only numerical data to later generate a smoothed bootstrap sample.
rus = RandomUnderSampler(random_state=42)


In [ ]:
#OVERSAMPLING
X_final_train_resampled, y_final_train_resampled = ros.fit_resample(X_final_train, y_final_train)

#Produces error, SMOTE and ADASYN cannot handle missing values
#X_final_train_resampled, y_final_train_resampled = SMOTE().fit_resample(X_final_train, y_final_train)
#X_final_train_resampled, y_final_train_resampled = ADASYN().fit_resample(X_final_train, y_final_train)
## Suggests to look at impute https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html to deal with the missing values



#UNDERSAMPLING
#X_final_train_resampled, y_final_train_resampled = rus.fit_resample(X_final_train, y_final_train)

In [ ]:
print(y_final_train[:-1].sum()/len(y_final_train[:-1])*100) #0.9293% UTI after timeseriesflattener
print(y_final_train[:-1].sum()) #835 UTI
print(len(y_final_train[:-1]))  #89853 TOTAL


print(y_final_test[:-1].sum()/len(y_final_test[:-1])*100) #0.9548% UTI after timeseriesflattener
print(y_final_test[:-1].sum()) #378 UTI
print(len(y_final_test[:-1]))  #39591 TOTAL

#Resampled
print(y_final_train_resampled[:-1].sum()/len(y_final_train_resampled[:-1])*100) #50% UTI after resampling
print(y_final_train_resampled[:-1].sum()) #89018 UTI
print(len(y_final_train_resampled[:-1]))  #178037 TOTAL


In [ ]:
y_final_train['outc_UTI_within_3_days_maximum_fallback_0_dichotomous']
y_final_train_resampled['outc_UTI_within_3_days_maximum_fallback_0_dichotomous']

In [ ]:
#Class weights

from sklearn.utils import class_weight

y_true_train = []

for i in y_final_train['outc_UTI_within_3_days_maximum_fallback_0_dichotomous']:
    y_true_train.append(int(i))

weight_dict = {0:1, 1:100}

class_weights2 = class_weight.compute_sample_weight(class_weight='balanced', y = y_final_train['outc_UTI_within_3_days_maximum_fallback_0_dichotomous'])
class_weights3 = class_weight.compute_sample_weight(class_weight='balanced', y = y_final_train_resampled['outc_UTI_within_3_days_maximum_fallback_0_dichotomous'])
#class_weights2 = class_weight.compute_sample_weight(class_weight= weight_dict, y = y_true_train)

In [ ]:
#print(*class_weights2, sep=",")

print(np.unique(class_weights2))
print(np.unique(class_weights3))


In [ ]:
import torch 

dtype = torch.float

In [ ]:
X_final_train[["pred_brain_injury"]] = X_final_train[["pred_brain_injury"]].astype("category")
X_final_test[["pred_brain_injury"]] = X_final_test[["pred_brain_injury"]].astype("category")
print(X_final_train.dtypes)
print(X_final_test.dtypes)

X_final_train_resampled[["pred_brain_injury"]] = X_final_train_resampled[["pred_brain_injury"]].astype("category")
print(X_final_train_resampled.dtypes)


In [ ]:
xgb_model = XGBClassifier(objective="binary:logistic", random_state=42, enable_categorical = True, tree_method = 'hist', reg_alpha = 100)

In [ ]:
from sklearn.metrics import accuracy_score

# fit!
#xgb_model.fit(X_final_train, y_final_train)
xgb_model.fit(X_final_train, y_final_train, sample_weight = class_weights2)

#Resampled
#xgb_model.fit(X_final_train_resampled, y_final_train_resampled, sample_weight = class_weights3)

# Evaluate
#print(cross_val_score(xgb_model, X_final_train, y_final_train, cv = 5, scoring = 'roc_auc'))

print(cross_val_score(xgb_model, X_final_train_resampled, y_final_train_resampled, cv = 5, scoring = 'roc_auc'))

#Reorder columns so they are in the correct order, otherwise I get an error 
#List of column names
cols_when_model_builds = xgb_model.get_booster().feature_names

#Reordering the columns of the dataframe 
X_final_test = X_final_test[cols_when_model_builds]


#Testing on actual new data

# make predictions
y_pred = xgb_model.predict(X_final_test)  
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_final_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
#Accuracy: 99.05% becuase it predicts 0 every time......



In [ ]:
#TJEK OM JEG HAR CLASS 6 OPGAVEN FRA NLP-KURSUS
#Har jeg ikke, slettet fra U-cloud og lå aldrig lokalt :( evt spørg Anna eller en anden fra årgangen under???

from LSTM import RNN
from torch import nn

embeddings = df_sentence_train

In [ ]:
# FloatTensor containing pretrained weights
#weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
#embedding = nn.Embedding.from_pretrained(weight)

In [ ]:
#Måske Ross' LSTM ikke er god, lader til at den vil lave embeddings selv... Jeg skal jo bare bruge en der kan læse det df jeg allerede har...
#embeddings = nn.Embedding.from_pretrained(X_final_train_resampled)

In [ ]:
LSTM_model = RNN(3, embeddings, 2)  #Youtube tutorial calls output_dim => input_len, hidden_dim_size => hidden_size. ALso has no embedding and no fc, but has "output_layer".  NB has input_len = 28 and sequence_len = 28, maybe corresponds to pixels in an image (mNist)

# print(LSTM_model) 

# #LSTM_model = RNN(output_dim: int, embedding_layer: nn.Embedding, hidden_dim_size: int)
# #What is output_dim? Is it the amount of predictions, or maybe the amount of possible prediction values????
# #Embedding layer seems like saved word_embeddings from earlier, do I need to seperate them from the other predictors and save as an nn.Embedding object?? 
# # hidden_dim_size? The dimension of the hidden layer? Or the amount of hidden layers?

# #Might need to include an optimizer in LSTM.py.... right now it will not optimize weights and biases. I actually cannot see the weights and biases in Ross' code, so I am a bit confused.¨
# # 10:48 ---> https://www.youtube.com/watch?v=FHdlXe1bSe4&t=840s

In [ ]:
print(f'Accuracy: {cross_val_score(xgb_model, X_final_train_resampled, y_final_train_resampled, cv = 5, scoring = "accuracy")}')
print(f'ROC-AUC: {cross_val_score(xgb_model, X_final_train_resampled, y_final_train_resampled, cv = 5, scoring = "roc_auc")}')
print(f'F1-score: {cross_val_score(xgb_model, X_final_train_resampled, y_final_train_resampled, cv = 5, scoring = "f1")}') #Should be possible to change the weights of f1, so it favors recall over precision. Would be relevant in my case as it is more important to have fewer false-negatives, than false-positives (more worried about MISSING true cases than mis-diagnosing)

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(f"True Positives: {TP}. False Positives: {FP}. True negatives: {TN}. False negatives: {FN}.")

In [ ]:
y_true = []

for i in y_final_test['outc_UTI_within_3_days_maximum_fallback_0_dichotomous']:
    y_true.append(int(i))

In [ ]:
y_true

In [ ]:
print(perf_measure(y_true, predictions))

from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

print(f'Precision = {average_precision_score(y_true, predictions)}')
print(f'Recall = {recall_score(y_true, predictions)}')
print(f'F1 score = {f1_score(y_true, predictions)}')
print(f'ROC-AUC score = {roc_auc_score(y_true, predictions)}') #0.524 that is predictive ability is not better than random guessing



In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    xgb_model, X_final_test, y_final_test, name="XGBoost") #, plot_chance_level=True)
    
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
print(*y_pred, sep=",")